In [14]:
#CTE de mejores vendedores, vendedores que mas generan ingresos
from src.db import run_sql
query= """WITH vendedor_top AS (
    SELECT
        SalesPersonID,
        SUM(TotalPrice) AS Ingreso_Generado
    FROM sales
    GROUP BY SalesPersonID
)
SELECT
    vt.SalesPersonID,
    vt.Ingreso_Generado,
    e.FirstName,
    e.LastName
FROM vendedor_top AS vt
  INNER JOIN employees AS e
    ON vt.SalesPersonID = e.EmployeeID
ORDER BY vt.Ingreso_Generado DESC
LIMIT 5;"""

df_customers = run_sql(query)
df_customers 

,SalesPersonID,Ingreso_Generado,FirstName,LastName
0,23,665091.0,Janet,Flowers
1,22,631158.0,Tonia,Mc Millan
2,21,618744.0,Devon,Brewer
3,20,569220.0,Shelby,Riddle
4,19,538954.0,Bernard,Moody


Vista creada para ver las ventas por mes

Nota: El codigo de la creación de la vista se encuentra en la carpeta "sql" bajo el nombre de View

In [ ]:
#UTILIZA LA VISTA QUE INDICA EL INGRESO POR MES
from src.db import run_sql
query= """ 
SELECT Mes, TotalMensual FROM ventas_total_mes
ORDER BY Mes ASC;"""

df_customers = run_sql(query)
df_customers 

,Mes,TotalMensual
0,1,741138.0
1,2,654883.0
2,3,728428.0
3,4,698180.0
4,5,724137.0
5,6,718858.0
6,7,708201.0
7,8,728896.0
8,9,709832.0
9,10,717826.0


Función de ventana que utiliza ROW_NUMBER(), donde mostrara la posición del ranking en la que se encuentra el mes según el total vendido, donde se ve que el mes 3(marzo) se encuentra en el TOP 1 y el mes de Dicimebre, el mes donde el ingreso fue el más bajo del año. Además se agrego la función LAG() para comparar las ventas con el mes anterior y como extra se añadio el crecimiento (o caida) con respecto al mes anterior

In [16]:
from src.db import run_sql

query = """
WITH ventas_base AS (
    SELECT
      Año,
      Mes,
      TotalMensual,
      LAG(TotalMensual) OVER (
        PARTITION BY Año 
        ORDER BY Mes
      ) AS TotalAnterior
    FROM ventas_total_mes
)
SELECT
  vb.Año,
  vb.Mes,
  vb.TotalMensual,
  vb.TotalAnterior,
  CASE
    WHEN vb.TotalAnterior IS NULL
      THEN NULL
    ELSE
      ROUND(
        (vb.TotalMensual - vb.TotalAnterior)
        / vb.TotalAnterior
        * 100
      , 2)
  END AS CrecimientoPct,
  ROW_NUMBER() OVER (
    ORDER BY vb.TotalMensual DESC
  ) AS RowNum
FROM ventas_base AS vb
ORDER BY vb.Año, vb.Mes;
"""
df = run_sql(query)
df


,Año,Mes,TotalMensual,TotalAnterior,CrecimientoPct,RowNum
0,2025,1,741138.0,NaN,NaN,1
1,2025,2,654883.0,741138.0,-11.64,11
2,2025,3,728428.0,654883.0,11.23,3
3,2025,4,698180.0,728428.0,-4.15,10
4,2025,5,724137.0,698180.0,3.72,4
5,2025,6,718858.0,724137.0,-0.73,5
6,2025,7,708201.0,718858.0,-1.48,9
7,2025,8,728896.0,708201.0,2.92,2
8,2025,9,709832.0,728896.0,-2.62,8
9,2025,10,717826.0,709832.0,1.13,6


Esta función ejecuta el procedimiento almacenado llamado "registro_productos", el cual funciona para almacenar de manera simple nuevos productos para vender

In [1]:
from src.db import execute_sql

query = """
CALL registro_productos(
    9998,
    'Chocolinas',
    200,
    5,
    'Low',
    '2024-06-02',
    'Durable',
    'Unknown',
    200
);
"""
execute_sql(query)


Probamos que funciona el procedimiento almacenado llamando a la tabla productos con el nuevo producto "Chocolinas"

In [17]:
from src.db import run_sql
query= """select ProductID, ProductName, Price from products
where ProductName='Chocolinas';"""
df_customers = run_sql(query)
df_customers 

,ProductID,ProductName,Price
0,9998,Chocolinas,200.0


Trigger de seguridad el cual muestra si el calculo realizado de Total Price se encuentra mal realizado, lo que evita errores o cargas fraudulentas. Se puede crear una tabla para almacenar estos registros erroneos o fraudulentos. Donde en el caso de que este mal la función muestra un error que dice 'Error: TotalPrice no coincide con Quantity * Precio - Discount.'

Nota: el trigger creado se encuentra en la carpeta sql


In [5]:
#Vamos a probarlo (caso de que se esta insertando una fila de Sales con el calculo del Total Price)
from src.db import execute_sql

query = """INSERT INTO sales (
    SalesID, 
    SalesPersonID, 
    CustomerID, 
    ProductID, 
    Quantity, 
    Discount, 
    TotalPrice,            
    SalesDate, 
    TransactionNumber
)
VALUES (
    99999, 
    1, 
    33133, 
    12, 
    2, 
    0.1,
    21.6,                
    '2025-03-15 12:00:00', 
    'TX-TEST-0001'
);"""

Se muestra el Indice creado, en la tabla Sales especificamente en la columna SaleaDate.

In [2]:
from src.db import run_sql

# 1. “SHOW INDEX” recupera todos los índices de la tabla `sales`
df_indexes = run_sql("SHOW INDEX FROM sales;")
df_indexes


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,sales,0,PRIMARY,1,SalesID,A,50000,None,None,,BTREE,,,YES,None
1,sales,1,idx_sales_salesperson,1,SalesPersonID,A,22,None,None,,BTREE,,,YES,None
2,sales,1,idx_sales_customer,1,CustomerID,A,38896,None,None,,BTREE,,,YES,None
3,sales,1,idx_sales_product,1,ProductID,A,448,None,None,,BTREE,,,YES,None
4,sales,1,idx_sales_salesdate,1,SalesDate,A,49432,None,None,YES,BTREE,,,YES,None
5,sales,1,idx_sales_productid,1,ProductID,A,453,None,None,,BTREE,,,YES,None
